In [12]:
%run ./common.ipynb

from dotenv import load_dotenv
load_dotenv()

api_token=os.getenv('JIRA_TOKEN')

if not api_token:
    raise ValueError("No JIRA API token found. Check your .env file / environment variable.")


global logger
logger = setup_logger(logging.INFO)
logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

2025-01-22 16:50:09,026 - root - INFO - =======================================
2025-01-22 16:50:09,027 - root - INFO - Script start accumulating data from JIRA


In [13]:

# run_proj = sys.argv[1]
run_proj ='ALL'


partitionCnt=5
filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/Timesheet/'

# filepath='../output/TimeSheet'
importfile='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/AllProjects/transition_history_Story.csv'

issueLists = [
            # {'issueType':"Portfolio Initiative",'partitionCnt':3},
            # {'issueType':"Product Initiative", 'partitionCnt':3},
            # {'issueType':"Epic",'partitionCnt':partitionCnt},
            {'issueType':"Story",'partitionCnt':partitionCnt}
            ]
             
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
# Jira ID 
# Name
# Planview Portfolio ID
# Work Description
# IssueType
# Work State
def portoFolioDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.issuetype.name',
                             'fields.status.name','fields.customfield_32307'
                         #  'fields.description',
        ]).rename(columns={
        'key':'JiraID',
        'fields.summary':'Name',
        'fields.customfield_32307':'Planview Portfolio ID',
        # 'fields.description':'Work Description',
        'fields.issuetype.name':'IssueType',
        'fields.status.name':'Work State'})

In [15]:
# Name
# ParentLink
# Work Description
# Jira ID
# IssueType
# Start date
# End date
# Work State
def productInitDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.customfield_12823',
                             'fields.issuetype.name','fields.status.name',
                            #  'fields.description',
                            'fields.project.name'
                            ]).rename(columns={
                            'key':'JiraID',
                            'fields.summary':'Name',
                            'fields.customfield_12823': 'ParentLink',
                            # 'fields.description':'Work Description',
                            'fields.issuetype.name':'IssueType',
                            'fields.status.name':'Work State', 'fields.project.name':'projectName'})

In [16]:
# Name ✅
# ParentLink ✅
# Work Description ❌
# Jira ID ✅
# IssueType ✅
# Start date ✅
# End date ✅
# Banner ✅
# Region ✅
# T-shirt Size ✅
# Epic Work Type ✅
# Work State ✅
# In-Progress DateDate ⏳
# Completed Date ⏳
def epicDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.customfield_12823',
                             'fields.issuetype.name','fields.status.name','fields.customfield_12305',
                             'fields.customfield_31400.value', 'fields.project.name', 
                             'fields.customfield_13800', 'fields.customfield_13801','fields.project.key','fields.customfield_10207.value',
                             'fields.customfield_17210','fields.customfield_17211'
                            #  'fields.description',
        ]).rename(columns={
        'fields.summary':'Name',
        'key':'JiraID',
        'fields.customfield_12823': 'ParentLink',
        # 'fields.description':'Work Description',
        'fields.customfield_12305':'Banner',
        'fields.issuetype.name':'IssueType',
        'fields.status.name':'Work State',
        'fields.project.name':'ProjectName',
        'fields.project.key':'ProjectId',
        'fields.customfield_31400.value': 'Epic Type',
        'fields.customfield_13800': 'Target start',
        'fields.customfield_13801': 'Target end',
        'fields.customfield_10207.value':'T-Shirt size',
        'fields.customfield_17210':'Affected Banners',
        'fields.customfield_17211':'Affected Geographies'})

In [17]:
# Name ✅
# ParentLink ✅
# Work Description ❌
# Jira ID ✅
# IssueType ✅
# Story Estimate ✅
# Story Work Type ✅
# Work State ✅
# In-Progress Date ✅
# Completed Date ✅
# Jira Project Name ✅
def StoryDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.customfield_10006',
                             'fields.customfield_10002',
                             'fields.issuetype.name','fields.status.name',
                             'fields.project.name','fields.customfield_31800.value','fields.project.key'
         #  'fields.description',
        ]).rename(columns={
        'fields.summary':'Name',
        'fields.customfield_10006': 'ParentLink', #Epic Link
      # 'fields.description':'Work Description',
        'fields.customfield_31800.value':'story type',
        'key':'JiraID',
         'fields.issuetype.name':'IssueType',
        'fields.customfield_10002':'StoryPoint',
        'fields.status.name':'Work State',
        'fields.project.name':'ProjectName',
        'fields.project.key':'ProjectId',})

In [18]:
def getDates(key,state):
    dates=''
    filterdata = importTransactionFile[importTransactionFile['key']==key]
    if state == 'close':    
     state= filterdata[filterdata['to'].isin(close_state)]
    else:
     state= filterdata[filterdata['to'].isin(development_state)]
    
    if state is not None and not state.empty:
        dates= state['created_at'].values[0]

    return dates

In [19]:
def data_clean(dataframe, issueType):
    if dataframe is not None and not dataframe.empty:
       if issueType == 'Portfolio Initiative':
         dataframe = portoFolioDataframe(dataframe)
       elif issueType == 'Product Initiative':
         dataframe = productInitDataframe(dataframe)
       elif issueType == 'Epic':
          dataframe = epicDataframe(dataframe)
          dataframe['Affected Banners']=dataframe['Affected Banners'].apply(lambda x: getBaselineLst(x) )
          dataframe['Affected Geographies']=dataframe['Affected Geographies'].apply(lambda x: getBaselineLst(x) )
          # dataframe['ProjectName'] = dataframe['ProjectName'].str.replace('&', '%26')
          header = list(dataframe.columns)
          
          if 'T-Shirt size' not in header:
            dataframe['T-Shirt size'] =""
            # dataframe = dataframe.reindex(columns=header, fill_value="")
          
          if 'Epic Type' not in header:
            dataframe['Epic Type'] =""

       elif issueType == 'Story':
          dataframe = StoryDataframe(dataframe)
          dataframe['In-Progress Date'] = dataframe['JiraID'].apply(lambda x: getDates(x,'inProgress'))
          dataframe['Completed Date'] = dataframe['JiraID'].apply(lambda x: getDates(x,'close'))
          # dataframe['ProjectName'] = dataframe['ProjectName'].str.replace('&', '%26')

          # header = list(dataframe.columns)
          # if 'In-Progress Date' not in header:
          #   dataframe['In-Progress Date'] =""
          # if 'Completed Date' not in header:
          #   dataframe['Completed Date'] =""
    return dataframe


In [20]:
def fetch( project, issueType):
    global fetch_df 
    logger.info(f"Fetch {issueType}...")

    # if issueType == 'Story':
    #     attribute =' &expand=projects.issuetypes.fields,changelog'
    # else:
    attribute =' '
    development_state_Lst = '", "'.join(development_state)
    deployment_state_Lst = '", "'.join(deployment_state)
    qa_state_Lst = '", "'.join(qa_state)

    liststr = '"{0}","{1}","{2}"'.format(development_state_Lst, deployment_state_Lst,qa_state_Lst )
    

    searchQry=f'?jql= project in ({project}) and issueType="{issueType}" and status in ({liststr}) and updated > -30d'

    # if cond_flg == True: 
    #     searchQry=f"{searchQry} and status not in (Cancelled, Done)  {attribute}"
    # else:
    searchQry=f"{searchQry} {attribute}"

    fetch_df = getDataSet(searchQry, project, issueType, api_token )

    return data_clean(fetch_df,issueType )    
    

In [21]:
def getChangeLog(element):
    changelog_Rec = [] 
    for history in element:
        created_at = history.get('created',None)
        for item in history['items']:
            if item['field'] =='status' :
                changelog_Rec.append({
                    'created_at': created_at,
                    'field': item['field'],
                    'from': item['fromString'],
                    'to': item['toString'],
                })
    return changelog_Rec

In [22]:
# clean folder
csv_files = Path(filepath).glob("*_working.csv")
for file in csv_files:
    filename=file.name
    os.remove(os.path.join(filepath, filename))

global importTransactionFile 
importTransactionFile = pd.read_csv(importfile)

# strings =pd.Series(["PCR"])
strings = projectList(api_token)

for issueList in issueLists:
    split_string_list = split_list(strings, issueList['partitionCnt'])
    filtered_list = [series for series in split_string_list if not series.empty]
    for partition in filtered_list:
        logger.warning(f"processing %s - %s",partition.values[0],issueList['issueType'])

        df=fetch(', '.join(['"{}"'.format(value) for value in partition]),issueList['issueType'])
        if df is not None and not df.empty:
            try:     
                    output_path = os.path.join(filepath, f"{issueList['issueType']}_working.csv")            
                    # df.to_csv(output_path ,mode='a', header=not os.path.exists(output_path), index=False)
                    append_to_csv(df,output_path)
            except Exception as e:  
                logger.error(f"Failed to upload files to {filepath}: {e}")
                pass
                
            logger.info("Move working files to current files")
            # Delete the old DataFrame 
            del(df)

            # Perform garbage collection                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
            gc.collect()
            # break
    clean_folder(filepath,"_working.csv")

del(importTransactionFile)
# Perform garbage collection                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
gc.collect()
logger.warning(f"Script ended accumulating data from JIRA ")
logger.warning("=======================================")


2025-01-22 16:50:09,546 - root - INFO - Getting report results...
2025-01-22 16:50:09,546 - root - INFO - https://jira.footlocker.com/rest/api/2/project
2025-01-22 16:50:10,424 - root - INFO - <Response [200]>
2025-01-22 16:50:10,425 - root - INFO - Report results received...
2025-01-22 16:50:10,425 - root - INFO - HTTP 200 - OK
2025-01-22 16:50:10,428 - root - WARNING - processing TEST - Story
2025-01-22 16:50:10,429 - root - INFO - Fetch Story...
2025-01-22 16:50:10,429 - root - INFO - Getting report results...
2025-01-22 16:50:10,429 - root - INFO - https://jira.footlocker.com/rest/api/2/search?jql= project in ("TEST", "CCEP", "CHAT", "COMPRODUCT", "AEMSC", "AWFM", "AII2", "AT", "BAE", "BOT", "BRWSE", "CHSC", "CNC", "CE", "CPENG", "CCOE", "CSP", "CA", "CMST", "CROHR", "CROPS", "COBUSS", "CEA", "CCL", "CEO", "CSZI", "CXCD", "INTEL") and issueType="Story" and status in ("In Development", "Code Review", "On Hold", "In Progress", "Dev Blocked", "Deploy to Staging", "Dev Review", "Blocke